In [1]:
!cd

D:\Ansys Simulations\Project\2D


# Data Pre-Processing

This notebook aims to, given the data exported from ANSYS, create new files that can be used in a convolutional neural network. The current idea on how to handle this is as follows:

#### Concatenating
The input data needs to be concatenated in one file with all nodes

#### Data Scaling
The input and output data have to be somehow scaled to ensure that the program can generalize to more scenarios. In an usual convolutional neural network, which usually deals with pixel-wise image data, this is simple, as all the values to be scaled are in the range from 0 to 255. This is not the case for this problem as the scale can vary arbitrarily from infinitesimals to very large numbers. What can be done, perhaps, is to create Pi groups to nondimensionalize the data, and then decide on a relevant non dimensional value to divide the data by to scale it. Another approach would be to use a relevant engineering value, such as the yield strength in our scenario, to scale it, however that requires previous engineering knowledge that we do not want to assume we have.

#### Shaping
After the scaling, this data has to be put in a format that can be used for a convolutional neural network. The kernel might have to be modified to travel between nodes instead of a grid, or maybe the data itself has to be transformed to be a continuous field which then could be split in a grid and a more usual kernel used to handle it. Yet a third option is to simply  leave the data as is and split it in a grid anyway. This last approach would require the grid to be such that it catches all nodes, so the smallest element size controls the grid size.

In [87]:
## Imports
import os
from pathlib import Path
import traceback
import re

# Concatenating the Input Data

In [5]:
%cd D:\Ansys Simulations\Project\2D\data

D:\Ansys Simulations\Project\2D\data


In [9]:
os.listdir()

['v12_test']

In [11]:
%cd D:\Ansys Simulations\Project\2D\data\v12_test

D:\Ansys Simulations\Project\2D\data\v12_test


In [12]:
os.listdir()

['data_dir_1',
 'data_dir_10',
 'data_dir_100',
 'data_dir_101',
 'data_dir_102',
 'data_dir_103',
 'data_dir_11',
 'data_dir_12',
 'data_dir_13',
 'data_dir_14',
 'data_dir_15',
 'data_dir_16',
 'data_dir_17',
 'data_dir_18',
 'data_dir_19',
 'data_dir_2',
 'data_dir_20',
 'data_dir_21',
 'data_dir_22',
 'data_dir_23',
 'data_dir_24',
 'data_dir_25',
 'data_dir_26',
 'data_dir_27',
 'data_dir_28',
 'data_dir_29',
 'data_dir_3',
 'data_dir_30',
 'data_dir_31',
 'data_dir_32',
 'data_dir_33',
 'data_dir_34',
 'data_dir_35',
 'data_dir_36',
 'data_dir_37',
 'data_dir_38',
 'data_dir_39',
 'data_dir_4',
 'data_dir_40',
 'data_dir_41',
 'data_dir_42',
 'data_dir_43',
 'data_dir_44',
 'data_dir_45',
 'data_dir_46',
 'data_dir_47',
 'data_dir_48',
 'data_dir_49',
 'data_dir_5',
 'data_dir_50',
 'data_dir_51',
 'data_dir_52',
 'data_dir_53',
 'data_dir_54',
 'data_dir_55',
 'data_dir_56',
 'data_dir_57',
 'data_dir_58',
 'data_dir_59',
 'data_dir_6',
 'data_dir_60',
 'data_dir_61',
 'data_dir

In [13]:
%cd D:\Ansys Simulations\Project\2D\data\v12_test\data_dir_1

D:\Ansys Simulations\Project\2D\data\v12_test\data_dir_1


In [14]:
os.listdir()

['disp_test_1.txt',
 'force_test_1.txt',
 'named_selection_test_1.txt',
 'named_selection_test_2.txt',
 'named_selection_test_3.txt',
 'named_selection_test_4.txt',
 'named_selection_test_5.txt',
 'named_selection_test_6.txt',
 'solutions_test_1.txt']

So each directory contains `disp_test_*.txt`, `force_test_*.txt`, `named_delection_*.txt` and `solutions_test_*.txt`. 

The objective is to transform the  `disp_test_*.txt`, `force_test_*.txt` and all `named_delection_*.txt`s to a single file

Since forces and displacements are assigned randomly, for we are going to use a file with a large number of both to develop the transformation script. One such  direcctory  is directory 26, with 12 selections in total, 6 of each deformation and force.

In [51]:
%cd D:/Ansys Simulations/Project/2D/data/v12_test/data_dir_26

D:\Ansys Simulations\Project\2D\data\v12_test\data_dir_26


In [52]:
os.listdir()

['disp_test_26.txt',
 'force_test_26.txt',
 'named_selection_test_1.txt',
 'named_selection_test_10.txt',
 'named_selection_test_11.txt',
 'named_selection_test_12.txt',
 'named_selection_test_2.txt',
 'named_selection_test_3.txt',
 'named_selection_test_4.txt',
 'named_selection_test_5.txt',
 'named_selection_test_6.txt',
 'named_selection_test_7.txt',
 'named_selection_test_8.txt',
 'named_selection_test_9.txt',
 'solutions_test_26.txt']

In [84]:
current_folder = pathlib.Path.cwd()
current_file = Path("disp_test_26.txt")
print(type(current_file), current_file)


with open(current_file) as file:
    for line in file:
        print(line)

<class 'pathlib.WindowsPath'> disp_test_26.txt
Selection	[m]	0.000606167164195471	-0.000287891718267512	0

Selection 2	[m]	-0.000985739677013019	-0.00208731508313133	0

Selection 3	[m]	-0.000158543117864855	-0.000118606609351938	0

Selection 4	[m]	-8.00937423301898E-06	0.00032369480229358	0

Selection 5	[m]	-0.000427524893052407	0.000184740376266274	0

Selection 6	[m]	0.00142152849334207	-0.00110920936809426	0



In [54]:
## CREATES FOLDERS FOR NEW FILES

#using pathlib to create a path
data_directory_path = Path('D:/Ansys Simulations/Project/2D/data')
try:
    input_folder = Path('input')
    output_folder = Path('output')
    input_path = Path(data_directory_path, input_folder)
    output_path = Path(data_directory_path, output_folder)
    input_path.mkdir()
    output_path.mkdir()
except Exception:
    print("folders likely already exist")
    traceback.print_exc()

folders likely already exist


Traceback (most recent call last):
  File "<ipython-input-54-672ae1d13b42>", line 10, in <module>
    input_path.mkdir()
  File "D:\Anaconda3_2nd\envs\AI_FEA\lib\pathlib.py", line 1268, in mkdir
    self._accessor.mkdir(self, mode)
FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'D:\\Ansys Simulations\\Project\\2D\\data\\input'


In [55]:
print(data_directory_path)
print(input_path)

D:\Ansys Simulations\Project\2D\data
D:\Ansys Simulations\Project\2D\data\input


In [58]:
pathlib.Path.cwd()

WindowsPath('D:/Ansys Simulations/Project/2D/data/v12_test/data_dir_26')

In [88]:
print(current_folder.name)

data_dir_26


In [91]:
##Create function that gets the sample number from the directory name

def get_sample_number(filename):
    return re.search(r'\d+', filename).group(0)

print(get_sample_number(current_folder.name))

26


In [74]:
## create new file with the correct number in the input and output directories
print(current_file)
with open(current_file) as file:
    for line in file:
        print(line)
        
input_file = Path(input_path, 'input_26.txt')
print(input_file)
output_file = Path(input_path, 'output_26.txt')
print(output_file)

disp_test_26.txt
Selection	[m]	0.000606167164195471	-0.000287891718267512	0

Selection 2	[m]	-0.000985739677013019	-0.00208731508313133	0

Selection 3	[m]	-0.000158543117864855	-0.000118606609351938	0

Selection 4	[m]	-8.00937423301898E-06	0.00032369480229358	0

Selection 5	[m]	-0.000427524893052407	0.000184740376266274	0

Selection 6	[m]	0.00142152849334207	-0.00110920936809426	0

D:\Ansys Simulations\Project\2D\data\input\input_26.txt
D:\Ansys Simulations\Project\2D\data\input\output_26.txt
